# Handwritten digit classifier with LeNet-5

## About this notebook

This notebook allows you to recognize digits (i.e., numbers from 0 to 9) manually drawn on your screen.

## LeNet-5

Convolutional Neural Networks is the standard architecture of a neural network designed for solving tasks associated with images (e.g., image classification). Some of the well-known deep learning architectures for CNN are LeNet-5 (7 layers), GoogLeNet (22 layers), AlexNet (8 layers), VGG (16–19 layers), and ResNet (152 layers). 

For this project, we use LeNet-5, which has been successfully used on the MNIST dataset to identify handwritten-digit patterns. The LeNet-5 architecture is represented in the following image.

![screenshot](img/lenet.png)

## Data

The dataset used to train, validate and test the model, correpsond to the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset. 
It is composed by a training set of 60,000 examples, and a test set of 10,000 examples. 
The digits have been pre-processed to be size-normalized and centered in a fixed-size image of 28x28 pixels.

![screenshot](img/mnist.png)

## Imports

In [1]:
# General imports
import numpy as np ; np.random.seed(1) # for reproducibility
import os
import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import shutil
from datetime import datetime
from sklearn.model_selection import train_test_split
%matplotlib inline

# TensorFlow
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D 

## TensorFlow information

In [2]:
# Indicate the version of Tensorflow and whether it uses the CPU or the GPU
print("TensorFlow version:", tf.__version__)

if len(tf.config.list_physical_devices('GPU')) > 0:
    print("The GPU will be used for calculations.")
    
else:
    print("The CPU will be used for calculations.")

TensorFlow version: 2.4.0
The GPU will be used for calculations.


## Functions

In [ ]:
def download_data(OS, url, distination_path):
    
    ''' Download data from an URL and save it locally '''
    
    # Import dataset
    try:
        if OS == 'macOS' or OS == 'Linux':
            os.system("!wget --no-check-certificate " + url + " -O " + distination_path)
        elif OS == 'Windows':
            !curl.exe --output $distination_path --url $url
        else:
            raise Exception('Please, select a valid Operating System (i.e., Windows, macOS or Linux)')

    except Exception as e:    
        raise Exception('Something went wrong downloading the data!')
    

## 1. Import data 

### Raw data

In [ ]:
# Import images and labels from the MNIST database
(train_X, train_y), (test_X, test_y) = tf.keras.datasets.mnist.load_data()

## 2. Describe data

In [ ]:
# Describe data dimensions
print('Training images dimensions:', train_X.shape)
print('Training labels size:', train_y.shape[0])
print('Test images dimensions:', test_X.shape)
print('Test labels size:', test_y.shape[0])

In [ ]:
# Number of pictures in the grid
nrows = 4
ncols = 4

# Set up matplotlib fig
longitude_image = 3 # Inches per picture
fig = plt.gcf()
fig.set_size_inches(ncols * longitude_image, nrows * longitude_image)

# Plot some examples
for i in range(nrows*ncols):  
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # Don't show axes (or gridlines)
    plt.imshow(train_X[i], cmap=plt.get_cmap('gray'))
    plt.title('This is a ' + str(train_y[i]))

# Plot grid
plt.show()

## 3. Data Preprocessing

### Shuffle data

In [ ]:
# Import images and labels from the MNIST database
(train_X, train_y), (test_X, test_y) = tf.keras.datasets.mnist.load_data()

# Shuffle train data
permut = np.random.permutation(train_X.shape[0])
train_X = train_X[permut]
train_y = train_y[permut]

# Shuffle test data
permut = np.random.permutation(test_X.shape[0])
test_X = test_X[permut]
test_y = test_y[permut]

### Reshape

In [ ]:
# Reshape images to include the channels
train_X = train_X.reshape(train_X.shape + (1,))
test_X = test_X.reshape(test_X.shape + (1,))

# Describe data dimensions
print('Training images dimensions:', train_X.shape)
print('Test images dimensions:', test_X.shape)

### Normalize

In [ ]:
# Normalize
train_X = train_X / 255.
test_X = test_X / 255.
# train_X = train_X.astype('float32')
# test_X = test_X.astype('float32')                

### Convert labels to One-hot

In [ ]:
# Converting Labels to one hot encoded format
train_y_one_hot = to_categorical(train_y)
test_y_one_hot = to_categorical(test_y)

### Create validation set

In [ ]:
# Create validation set
X_train, X_val, y_train, y_val = train_test_split(train_X, 
                                                  train_y_one_hot, 
                                                  test_size=0.05, 
                                                  random_state=1)

In [ ]:
# Describe data dimensions
print('Training images dimensions:', X_train.shape)
print('Training labels size:', y_train.shape[0])
print('Validation images dimensions:', X_val.shape)
print('Validation labels size:', y_val.shape[0])
print('Test images dimensions:', test_X.shape)
print('Test labels size:', test_y.shape[0])

## 4. Convolutional NN
The "output shape" column in the summary shows how the size of your feature map evolves in each successive layer. The convolution layers reduce the size of the feature maps by a bit due to padding, and each pooling layer halves the dimensions.

### Model

In [ ]:
# Create LeNet-5 architecture
lenet_5_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Print summary
lenet_5_model.summary()

### Compile

In [ ]:
# Compile model
lenet_5_model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

## 5. Train

In [ ]:
# Set seed for random functions
tf.random.set_seed(1)

# Fit the model
history = lenet_5_model.fit(
    x=X_train,
    y=y_train,
    batch_size=32,
    epochs=25,
    verbose=2,
    validation_data=(X_val, y_val)
)

In [ ]:
# Save model
lenet_5_model.save_weights('./model/wieghts_lenet_5.h5')
lenet_5_model.save('./model/digit_recognizer_lenet_5.h5')

## 6. Evaluating Accuracy and Loss for the Model

In [ ]:
# Get metrics on training and test data
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc)) 

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc, label='Training accuracy')
plt.plot(epochs, val_acc, label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.legend(loc=0)
plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.legend(loc=0)
plt.title('Training and validation loss')
plt.xlabel('Epochs')

## 7. Test images

In [ ]:
# Number of pictures in the grid
nrows = 4
ncols = 4

# Set up matplotlib fig
longitude_image = 3 # Inches per picture
fig = plt.gcf()
fig.set_size_inches(ncols * longitude_image, nrows * longitude_image)

# Plot some examples
for i in range(nrows*ncols):
    # Pre-process image
    x = test_X[i].reshape((1,) + test_X[i].shape)
    # Predict
    classes = lenet_5_model.predict(x)
    certainty = str(np.max(classes*100).round(1)) + '%'
    prediction = np.argmax(classes, axis=1)
    # Plot image
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # Don't show axes (or gridlines)
    plt.imshow(test_X[i], cmap=plt.get_cmap('gray'))
    plt.title('This is a ' + str(prediction[0]) + ' [' + certainty + ']')

# Plot grid
plt.show()


## 8. Use case

In [ ]:
# Directory with images to test
use_case_path = './data/use_case'

# Get names of pictures
use_case_names = os.listdir(use_case_path)

# Number of pictures in the grid
nrows = 4
ncols = 4

# Set up matplotlib fig
longitude_image = 3 # Inches per picture
fig = plt.gcf()
fig.set_size_inches(ncols * longitude_image, nrows * longitude_image)

# Get path to each image
use_case_pix = [os.path.join(use_case_path, fname) for fname in use_case_names[:nrows*ncols]]

# Plot some examples
for i, img_path in enumerate(use_case_pix):
    # Load image
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(28, 28), grayscale=True)
    # Pre-process image
    input_img = keras.preprocessing.image.img_to_array(img)
    input_img = input_img / 255.
    input_img = input_img.reshape((1,) + input_img.shape)
    # Predict
    classes = lenet_5_model.predict(input_img)
    certainty = str(np.max(classes*100).round(1)) + '%'
    prediction = np.argmax(classes, axis=1)
    # Plot image
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # Don't show axes (or gridlines)
    img = mpimg.imread(img_path)
    plt.imshow(img, cmap=plt.get_cmap('gray'))
    plt.title('This is a ' + str(prediction[0]) + ' [' + certainty + ']')

# Plot grid
plt.show()

In [ ]:
# Print time when finished
now = datetime.now()
print("Finished! At", now.strftime("%Y-%m-%d %H:%M:%S"))

In [36]:
import tkinter as tk
from PIL import Image, ImageDraw

class ImageGenerator:
    def __init__(self,parent,posx,posy,*kwargs):
        self.parent = parent
        self.posx = posx
        self.posy = posy
        self.sizex = 500
        self.sizey = 500
        self.b1 = "up"
        self.xold = None
        self.yold = None 
        self.drawing_area=tk.Canvas(self.parent,width=self.sizex,height=self.sizey)
        self.drawing_area.place(x=self.posx,y=self.posy)
        self.drawing_area.bind("<Motion>", self.motion)
        self.drawing_area.bind("<ButtonPress-1>", self.b1down)
        self.drawing_area.bind("<ButtonRelease-1>", self.b1up)
        self.button=tk.Button(self.parent,text="Done!",width=10,bg='white',command=self.save)
        self.button.place(x=self.sizex/7,y=self.sizey+20)
        self.button1=tk.Button(self.parent,text="Clear!",width=10,bg='white',command=self.clear)
        self.button1.place(x=(self.sizex/7)+80,y=self.sizey+20)

        self.image=Image.new("RGB",(200,200),(0,0,0))
        self.draw=ImageDraw.Draw(self.image)

    def save(self):
        use_case_path = './data/use_case'
        filename = './data/use_case/test.png'
        self.image.save(filename)

    def clear(self):
        self.drawing_area.delete("all")
        self.image=Image.new("RGB",(200,200),(0,0,0))
        self.draw=ImageDraw.Draw(self.image)

    def b1down(self,event):
        self.b1 = "down"

    def b1up(self,event):
        self.b1 = "up"
        self.xold = None
        self.yold = None

    def motion(self,event):
        if self.b1 == "down":
            if self.xold is not None and self.yold is not None:
                event.widget.create_line(self.xold,self.yold,event.x,event.y,smooth='true',width=3,fill='white')
                self.draw.line(((self.xold,self.yold),(event.x,event.y)),(0,128,0),width=3)

        self.xold = event.x
        self.yold = event.y

if __name__ == "__main__":
    root=tk.Tk()
    root.wm_geometry("%dx%d+%d+%d" % (400, 400, 10, 10))
    root.config(bg='black')
    ImageGenerator(root,10,10)
    root.mainloop()

In [37]:
from tkinter import *
import PIL
from PIL import Image, ImageDraw


def save():
    global image_number
    filename = f'image_{image_number}.png'   # image_number increments by 1 at every save
    image1.save('./data/use_case/' + filename)
    image_number += 1
    
def clear():
    cv.delete("all")
    #for item in cv.find_all():
    #    cv.delete(item)


def activate_paint(e):
    global lastx, lasty
    cv.bind('<B1-Motion>', paint)
    lastx, lasty = e.x, e.y


def paint(e):
    global lastx, lasty
    x, y = e.x, e.y
    cv.create_line((lastx, lasty, x, y), width=1)
    #  --- PIL
    draw.line((lastx, lasty, x, y), fill='black', width=1)
    lastx, lasty = x, y


root = Tk()

lastx, lasty = None, None
image_number = 0

cv = Canvas(root, width=640, height=480, bg='white')
# --- PIL
image1 = PIL.Image.new('RGB', (640, 480), 'white')
draw = ImageDraw.Draw(image1)

cv.bind('<1>', activate_paint)
cv.pack(expand=YES, fill=BOTH)

btn_save = Button(text="Save", command=save)
btn_save.pack()
btn_save = Button(text="Clear", command=clear)
btn_save.pack()

root.mainloop()